In [116]:
import pandas as pd
import numpy as np

In [117]:
bottle_csv_path = "/iliad/u/jmeribe/projects/vlm-scaffolding/tracking_results_apple_with_my_policies.csv"
bottle_csv_with_base = "/iliad/u/jmeribe/projects/vlm-scaffolding/tracking_results_bottle_with_vincent_policies.csv"

bottle_df = pd.read_csv(bottle_csv_path)

bottle_df

,Unnamed: 0,method,environment,success,num_envs,run_name,model_name,split,distribution
0,0,scripted,EnvApple-v0-phys-rand-with-cond,0.068359,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
1,1,scripted,EnvApple-v0-phys-rand,0.083984,1024,irrelevant,model_1999.pt,test,"beta(a=6.1, b=50.0)"
2,2,scripted,EnvApple-v0-phys-rand-with-cond,0.000977,1024,irrelevant,model_1999.pt,test,"beta(a=50, b=6.1)"
3,3,scripted,EnvApple-v0-phys-rand,0.013672,1024,irrelevant,model_1999.pt,test,"beta(a=50, b=6.1)"
4,4,scripted,EnvApple-v0-phys-rand-with-cond,0.045898,1024,irrelevant,model_1999.pt,test,"beta(a=50, b=50)"
...,...,...,...,...,...,...,...,...,...
95,95,scripted,EnvApple-v0-phys-rand,0.010742,1024,irrelevant,model_250.pt,train1,"beta(a=50, b=50)"
96,96,scripted,EnvApple-v0-phys-rand-with-cond,0.138672,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=1.9)"
97,97,scripted,EnvApple-v0-phys-rand,0.133789,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=1.9)"
98,98,scripted,EnvApple-v0-phys-rand-with-cond,0.298828,1024,irrelevant,model_250.pt,train1,"beta(a=2.5, b=6.8)"


In [118]:
# bottle_df_with_base = pd.read_csv(bottle_csv_with_base)

# General Performance: 
### conditioned policy narrowly wins
### original policy loses by 100%

In [119]:
valid_bottle_no_cond_df = bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand")]["success"]

general_no_cond = valid_bottle_no_cond_df.to_numpy().mean()
general_no_cond

0.11826171875

In [120]:
valid_bottle_cond_df = bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand-with-cond")]["success"]

general_cond = valid_bottle_cond_df.to_numpy().mean()
general_cond

0.12654296875

In [121]:
# bottle_df_vincent = bottle_df_with_base[(bottle_df_with_base['num_envs'] > 1000)]["success"]

# general_no_cond_vincent = bottle_df_vincent.to_numpy().mean()
# general_no_cond_vincent

# Performance Across Different Epochs
### conditioned policy loses 2/5

In [122]:
epoch_names = bottle_df["model_name"].unique()
epoch_names

array(['model_1999.pt', 'model_1750.pt', 'model_1250.pt', 'model_750.pt',
       'model_250.pt'], dtype=object)

In [123]:
epoch_perf_cond = []
epoch_perf_no_cond = []

for epoch in epoch_names:
    epoch_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand-with-cond") & (bottle_df["model_name"] == epoch)]["success"].to_numpy().mean()
    )
    epoch_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand") & (bottle_df["model_name"] == epoch)]["success"].to_numpy().mean()
    )

epoch_dict_cond = dict(zip(epoch_names, epoch_perf_cond))

epoch_dict_cond

{'model_1999.pt': 0.12646484375,
 'model_1750.pt': 0.13349609375,
 'model_1250.pt': 0.10986328125,
 'model_750.pt': 0.14228515625,
 'model_250.pt': 0.12060546875}

In [124]:
epoch_dict_un_cond = dict(zip(epoch_names, epoch_perf_no_cond))

epoch_dict_un_cond

{'model_1999.pt': 0.11396484375,
 'model_1750.pt': 0.114453125,
 'model_1250.pt': 0.1181640625,
 'model_750.pt': 0.14638671875,
 'model_250.pt': 0.09833984375}

In [125]:
wins = 0
games = 0

for key, value in epoch_dict_cond.items():
    wins += 1 if value > epoch_dict_un_cond[key] else 0
    games += 1

print(f"{wins}/{games}")

3/5


# Train and Test
### Conditioned policy wins on train loses on test

In [126]:
dataset_perf_cond = []
dataset_perf_no_cond = []

datasets = ["train1", "test"]

for dataset in datasets:
    dataset_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand-with-cond") & (bottle_df["split"] == dataset)]["success"].to_numpy().mean()
    )
    dataset_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand") & (bottle_df["split"] == dataset)]["success"].to_numpy().mean()
    )

dict(zip(datasets, dataset_perf_cond))

{'train1': 0.12828125, 'test': 0.1248046875}

In [127]:
dict(zip(datasets, dataset_perf_no_cond))

{'train1': 0.1205859375, 'test': 0.1159375}

# Distribution
### Conditioned policy loses 1/5

In [128]:
distribution_names = bottle_df["distribution"].unique()
distribution_names

array(['beta(a=6.1, b=50.0)', 'beta(a=50, b=6.1)', 'beta(a=50, b=50)',
       'beta(a=2.5, b=1.9)', 'beta(a=2.5, b=6.8)'], dtype=object)

In [129]:
distribution_perf_cond = []
distribution_perf_no_cond = []

for distribution in distribution_names:
    distribution_perf_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand-with-cond") & (bottle_df["distribution"] == distribution)]["success"].to_numpy().mean()
    )
    distribution_perf_no_cond.append(
        bottle_df[(bottle_df['num_envs'] > 1000) & (bottle_df["environment"] == "EnvApple-v0-phys-rand") & (bottle_df["distribution"] == distribution)]["success"].to_numpy().mean()
    )

distribution_dict_cond =  dict(zip(distribution_names, distribution_perf_cond))

In [130]:
distribution_dict_un_cond = dict(zip(distribution_names, distribution_perf_no_cond))

In [131]:
wins = 0
games = 0

for key, value in distribution_dict_cond.items():
    wins += 1 if value > distribution_dict_un_cond[key] else 0
    games += 1

print(f"{wins}/{games}")

4/5


# Conclusions
1. On basically every hyperparameter set when we don't condition the policy on the physical parameters of object being manipulated we perform better.

2. This doesn't make sense as we would expect the unconditioned policy to act as a lower bound for the performance of the conditioned policy.

3. However, when we compare average train and test performance we see that the conditioned policy performs better on train set and worse on the test set.

4. Indicating, that the conditioned policy is probably overfitting and may just be harder to train, given the setup.

5. Addtionally, the slim margins and generally low performance that we observe indicate that the scaffolding framework is not effective for tasks with high entropy physical property distributions or tasks where physicaly properties are important for performance, because it does not seem like it is effectively using them in this case.